In [ ]:
! pip install -U langgraph langsmith
! pip install langchain_community
! pip install langchain_openai

In [1]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END ,state
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

In [ ]:
 

# Configuration de la clé API
os.environ['OPENAI_API_KEY'] = "sk-proj-0N4HruNnPRcA"


# Création du modèle LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Step-Back Prompting: A Strategic Approach to Strengthen LLM Reasoning

Step-Back Prompting, introduced by Zheng et al. (2023c), is an innovative method that modifies the classic Chain-of-Thought reasoning approach by adding an essential preliminary phase.

## How Does Step-Back Prompting Work?

Imagine a mathematician faced with a complex problem. Before diving into detailed calculations, they would first take a step back to identify general concepts and relevant theoretical principles.

Step-Back Prompting reproduces this intellectual approach in two distinct phases:

1. **Generic preliminary question**: Before addressing the specific problem, the LLM is asked to reflect on high-level concepts, general principles, or fundamental facts related to the problem domain.

2. **Detailed reasoning**: Only after this conceptual contextualization step does the model engage in detailed reasoning to solve the specific problem.

## Why Is This So Effective?

This approach offers considerable advantages by allowing the model to:

* Activate relevant knowledge before applying it
* Establish a solid conceptual framework to guide reasoning
* Avoid errors due to an overly direct and hasty approach
* Break down complex problems into theoretical and then practical components

The results are particularly impressive in:
* Mathematical reasoning problems
* Complex scientific questions
* Problems requiring the application of abstract principles to concrete cases

It's like teaching a student to take the time to understand the theory before tackling exercises.

This method has demonstrated significant improvements on several reasoning benchmarks for both PaLM2L and GPT-4 models, confirming its robustness and effectiveness across different LLM architectures.

The next time you ask an AI a complex question, try inviting it to first reflect on relevant general concepts before asking for the specific answer!

In [4]:
step_back_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    Generate Step-Back Prompting based on user questions.
    
    Step-Back Prompting is a modification of Chain-of-Thought (CoT) where the LLM is first asked to consider high-level concepts or principles before delving into detailed reasoning.
    
    This technique significantly improves performance on complex reasoning tasks by encouraging the model to first understand the broader context and applicable principles before addressing specific details.
    
    The Step-Back process includes the following steps:
    1. High-level abstraction: First, identify the general principles, concepts, or frameworks that are relevant to the question.
    2. Conceptual analysis: Explore how these high-level concepts apply to the specific problem.
    3. Detailed reasoning: Once the conceptual framework is established, work through the specific problem step by step.
    4. Answer the original question: Provide a final answer based on the analysis.
    
    Please structure your response in the following format:
    
    Step 1: [High-level concepts and principles]
    Step 2: [Application of these concepts to the problem]
    Step 3: [Step-by-step reasoning]
    Step 4: [Final Answer]
    
    Example:
    Question: "A store is having a 25% off sale. If an item originally costs $80, what is the final price after the discount and 7% sales tax?"
    
    Step 1: High-level concepts and principles:
    Before diving into calculations, let me consider the key concepts involved in this problem:
    - Percentage discounts: When an item is discounted by X%, we multiply the price by (100%-X%) or (1-X/100)
    - Sales tax: Tax is applied after discounts, and we add X% tax by multiplying by (100%+X%) or (1+X/100)
    - Order of operations: In retail pricing, discounts are applied first, then tax is calculated on the discounted amount
    
    Step 2: Application of these concepts:
    For this specific problem:
    - We need to apply a 25% discount to the original price of $80
    - Then we need to apply a 7% sales tax to the discounted price
    
    Step 3: Step-by-step reasoning:
    1. Calculate the discounted price:
       - Original price = $80
       - Discount rate = 25% = 0.25
       - Discount amount = $80 × 0.25 = $20
       - Discounted price = $80 - $20 = $60
       - Alternatively: Discounted price = $80 × (1 - 0.25) = $80 × 0.75 = $60
    
    2. Calculate the tax on the discounted price:
       - Discounted price = $60
       - Tax rate = 7% = 0.07
       - Tax amount = $60 × 0.07 = $4.20
    
    3. Calculate the final price:
       - Final price = Discounted price + Tax amount
       - Final price = $60 + $4.20 = $64.20
    
    Step 4: The final price of the item after applying the 25% discount and 7% sales tax is $64.20.
    """),
    ("human", "{question}"),
    ("assistant", """
    Step 1: High-level concepts and principles:
    [Identify and explain the general principles, theories, or frameworks that are relevant to the question. What domain knowledge is necessary to understand this problem at a conceptual level?]
    
    Step 2: Application of these concepts:
    [Explain how these high-level concepts specifically relate to the problem at hand. What aspects of these principles are most relevant? How do they provide a framework for solving the specific question?]
    
    Step 3: Step-by-step reasoning:
    [Now that the conceptual groundwork is laid, work through the problem methodically:
    1. [First step in the solution process]
    2. [Second step in the solution process]
    3. [Continue with additional steps as needed]
    ...]
    
    Step 4: [Provide a clear, concise answer to the original question, drawing directly from the reasoning process above]
    """)
])

For Step-Back Prompting, the most suitable questions are those that benefit from first establishing high-level conceptual foundations before diving into specific problem-solving. Here are types of questions that would be particularly well-suited to this approach:

### Scientific Problem Solving
- Physics problems requiring application of fundamental laws
- Chemistry questions involving reaction mechanisms or principles
- Biology problems relating to systemic processes or evolutionary principles
- Environmental science questions requiring multi-disciplinary understanding

### Mathematical Reasoning
- Word problems that map to specific mathematical concepts
- Proof-based questions in geometry or algebra
- Applied mathematics problems in real-world contexts
- Probability and statistics questions requiring conceptual clarity first

### Conceptual Analysis
- Philosophical questions requiring definitional clarity
- Questions involving ethical frameworks or principles
- Policy analysis requiring understanding of underlying theories
- Technical questions that benefit from domain knowledge

### Concrete Examples of Questions

1. "How would you calculate the trajectory of a projectile fired at a 45-degree angle with an initial velocity of 20 m/s?"
   - Step-back would first explore concepts of projectile motion, gravity, and kinematics

2. "What would be the most appropriate statistical test to determine if there's a significant difference between these two treatment groups?"
   - Step-back would establish statistical principles, hypothesis testing, and data type considerations

3. "How would you design a sustainable urban transportation system that reduces carbon emissions by 50%?"
   - Step-back would explore transportation system principles, sustainability concepts, and emission factors

4. "What would be the environmental impact of replacing 30% of beef consumption with plant-based alternatives?"
   - Step-back would establish concepts of carbon footprints, land use, agricultural systems, and nutritional requirements

5. "How should we approach the ethical question of data privacy in AI-driven healthcare systems?"
   - Step-back would first explore broader principles of medical ethics, data ownership, and stakeholder interests

Step-Back Prompting is particularly effective for these types of questions because:

- It activates relevant conceptual knowledge before application
- It establishes a solid theoretical foundation that guides the specific reasoning
- It helps avoid jumping to solutions before understanding the underlying principles
- It improves reasoning by connecting specific problems to general knowledge domains
- It mimics expert problem-solving approaches which typically involve contextualizing problems within broader knowledge frameworks

This approach is especially powerful when dealing with problems that require application of established principles or when proper framing of the problem is crucial to finding the correct solution path.

In [6]:
import pandas as pd
import re
from typing import Optional, TypedDict, Annotated
from langchain.schema import AIMessage, HumanMessage

# Function to generate the Step-Back prompt and extract the final answer (Step 4)
def generate_step_back_node(state):
    question = state['messages'][-1].content  # Get the last question
    prompt_value = step_back_prompt_template.invoke({"question": question})
    messages = prompt_value.to_messages()
    response = llm.invoke(messages)
    
    # Keep the full response for processing
    full_response = response.content
    
    # Extract the final answer from Step 4
    final_answer_pattern = r'Step 4:([\s\S]*?)(?=$|Step \d:)'
    final_answer_match = re.search(final_answer_pattern, full_response)
    
    if final_answer_match:
        final_solution = final_answer_match.group(1).strip()
    else:
        # Alternative pattern
        alternative_pattern = r'Step 4: \[Final Answer\]([\s\S]*?)(?=$|Step \d:)'
        alternative_match = re.search(alternative_pattern, full_response)
        
        if alternative_match:
            final_solution = alternative_match.group(1).strip()
        else:
            final_solution = "Final solution not found in the expected format."
    
    # Save both the question, final answer, and full processing to a text file
    with open('step_back_result.txt', 'w', encoding='utf-8') as f:
        f.write("=== QUESTION ===\n\n")
        f.write(question)
        f.write("\n\n=== FINAL ANSWER (STEP 4) ===\n\n")
        f.write(final_solution)
        f.write("\n\n=== COMPLETE PROCESSING (FOR REFERENCE) ===\n\n")
        f.write(full_response)
    
    # Return only the final answer in messages, plus the original question for reference
    return {
        "messages": [AIMessage(content=final_solution)],
        "question": question,
        "full_response": full_response
    }

# Definition of state
class State(TypedDict):
    messages: Annotated[list, add_messages]
    question: Optional[str]
    full_response: Optional[str]

# Graph creation
graph_builder = StateGraph(State)
graph_builder.add_node("generate_step_back", generate_step_back_node)

# Graph configuration
graph_builder.set_entry_point("generate_step_back")
graph_builder.add_edge("generate_step_back", END)
graph = graph_builder.compile()

# Usage example
inputs = {
    "messages": [HumanMessage(content="How would you calculate the trajectory of a projectile fired at a 45-degree angle with an initial velocity of 20 m/s?")],
    "question": None,
    "full_response": None
}

# Print the question at the beginning
print("QUESTION:")
print(inputs["messages"][0].content)
print("\nFINAL ANSWER (STEP 4):")

# Graph execution - only display Step 4
original_question = ""
for output in graph.stream(inputs):
    for key, value in output.items():
        if key == "generate_step_back":
            messages = value['messages']
            for message in messages:
                if isinstance(message, AIMessage):
                    print(message.content)
            
            # Store question for reference
            if "question" in value:
                original_question = value["question"]

# Print confirmation that results were saved
print("\n(Complete results saved in 'step_back_result.txt')")

QUESTION:
How would you calculate the trajectory of a projectile fired at a 45-degree angle with an initial velocity of 20 m/s?

FINAL ANSWER (STEP 4):
The trajectory of the projectile fired at a 45-degree angle with an initial velocity of 20 m/s results in a range of approximately 40.65 meters, a maximum height of approximately 10.1 meters, and a total time of flight of approximately 2.88 seconds.

(Complete results saved in 'step_back_result.txt')
